## Stock Sentiment Analysis using News Headlines

In [55]:
import pandas as pd

In [56]:
messages=pd.read_csv('bangla_sms_data.txt', sep='\t', names=["label", "messages"])

In [57]:
messages.head()

,label,messages
0,ham,"জুরং পয়েন্ট অবধি পাগল হওয়া, ক্রেজি .. কেবল ..."
1,ham,ঠিক আছে লার ... আপনি যদি ওনিকে ...
2,spam,1 21 মে 2005 এফএ কাপ ফাইনালের টিকিট জিতে 2 উইক...
3,ham,ইউ দুন এত তাড়াতাড়ি বলে ... ইউ সি ইতিমধ্যে ত...
4,ham,"নাহ, আমি মনে করি না তিনি ইউএসএফের কাছে যান, য..."


In [73]:
from bltk.langtools import remove_stopwords
from bltk.langtools import Tokenizer
from bltk.langtools import UgraStemmer
import re

tokenizer = Tokenizer()
stemmer = UgraStemmer()


corpus = []
y_val = []
for i in range(0, len(messages)):
    review = messages['messages'][i]
    y_val_temp = messages['label'][i]
    review = "".join(i for i in review if i in ["।"] or 2432 <= ord(i) <= 2559 or ord(i)== 32)
    review =" ".join(review.split())
    
    review = tokenizer.word_tokenizer(review)
    #print(review)
    while("" in review) : 
        review.remove("") 
    review = remove_stopwords(review, level='hard')
    #print(review)
    review = stemmer.stem(review)
    #print(review)
    if(review==None):
        continue
    review = ' '.join(review)
    
    corpus.append(review)
    y_val.append(y_val_temp)
    


print("finished preprocessing")
# Creating the Bag of Words model
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=1800)
X = cv.fit_transform(corpus).toarray()

y=pd.get_dummies(y_val)#(messages['label'])
y = y.iloc[:,1].values

#X = X.reshape(X.shape[1:])
print(X.shape)
print(y.shape)
print(y)

# Train Test Split

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

# Training model using Naive bayes classifier

from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB().fit(X_train, y_train)

y_pred=spam_detect_model.predict(X_test)




/Users/fahadhasan/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator DictVectorizer from version 0.21.3 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/Users/fahadhasan/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.21.3 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/Users/fahadhasan/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator Pipeline from version 0.21.3 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


finished preprocessing
(1953, 1087)
(1953,)
[0 0 1 ... 0 0 0]


In [77]:
## Import library to check accuracy
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

In [78]:
matrix=confusion_matrix(y_test,y_pred)
print(matrix)
score=accuracy_score(y_test,y_pred)
print(score)
report=classification_report(y_test,y_pred)
print(report)

[[323  15]
 [ 17  36]]
0.9181585677749361
              precision    recall  f1-score   support

           0       0.95      0.96      0.95       338
           1       0.71      0.68      0.69        53

    accuracy                           0.92       391
   macro avg       0.83      0.82      0.82       391
weighted avg       0.92      0.92      0.92       391

